# Starlink-pywrapper.

The starlink-pywrapper package allows the calling of starlink command line tools in a pythonic way, and easy scripting using Starlink commands. Inside, it calls Starlink commands with subprocess, after setting up all of the environmental variables required. It also will read the output parameters from the $ADAM_DIR .sdf files and pack those up into a named tuple which is returned to the user. Please note that it is only running the Starlink tasks as external commands, so for those commands which produce a new file it will return the name of that file (along with any other output parameters).

## Dependencies
This package depends on the starlink-pyndf module to read the HDS files which hold the output of Starlinlk commands. This is unfortunately not available through PyPi (e.g. pip) currently, but can be downloaded and installed from:  https://github.com/timj/starlink-pyndf (installation instructions found at that site). Please note that a normal install of this will require you to set the LD_LIBRARY_PATH (Linux) or DYLD_LIBRARY_PATH (OSX) environmental variable to $STARLINK_DIR/lib .

This package also requires a working Starlink installation, which should be the same version of Starlink as that used to generate the module files inside the package (kappa.py etc.). If you have the environmental variable $STARLINK_DIR set to the location of your Starlink install before you run the package, then it will find Starlink automatically. If it does not, you can use the starlink.wrapper.change_starpath command to set the Starlink location after you have imported the package.

## Overview
The main Starlink command-packages supported are those used for analysis and data reduction of current JCMT instrumentation: KAPPA, SMURF, ATOOLS, Polpack, CUPID and CONVERT. There is also support for running ORAC-DR and PICARD recipes. 

 
## Getting started
 
First we need to setup logging: this package uses the standard python logger to give information about our commands.
 

In [1]:
import logging

In [2]:
logging.basicConfig(level=logging.INFO)

Now we can import the KAPPA wrapping module (starlink.kappa)

In [3]:
from starlink import kappa

INFO:starlink.wrapper:Using $STARLINK_DIR starlink at /star


The ```INFO:``` line of output above comes from the standard logger, and is showing us which STARLINK_DIR is being used. 

Now we can run commands from Starlink's KAPPA package like so:

In [4]:
statresults = kappa.stats('/home/sgraves/mask_trim.sdf')

The variable ```statresults``` returned from ```kappa.stats``` is a ```namedtuple``` object. You access its attributes by ```variable.<attribute>```.

In [5]:
statresults

    comp    b'DATA'
kurtosis    14.2191822912
   sigma    6.9888303341
maxcoord    [ -1.33963267e+00   2.05092512e-02   3.52668762e+02]
 minimum    1.0
 maximum    47.0
    mean    3.57111682266
  minpos    [54760 56801     1]
  minwcs    b'18:53:35.1, 1:11:23, 352.6688'
 numgood    20621
  numbad    547814
     ndf    b'/home/sgraves/mask_trim.sdf'
   total    73640.0
  numpix    568435
skewness    3.63018502818
   order    0
mincoord    [ -1.33698416e+00   2.07636185e-02   3.52668762e+02]
  maxwcs    b'18:52:58.7, 1:10:30, 352.6688'
  maxpos    [54883 56899     1]

In [6]:
statresults.mean

3.5711168226565015

If you need to programatically see all the fields in a results, you can look at the ```_fields``` attribute, which is a list of all the fields in the namedtuple.

In [7]:
statresults._fields

('comp',
 'kurtosis',
 'sigma',
 'maxcoord',
 'minimum',
 'maximum',
 'mean',
 'minpos',
 'minwcs',
 'numgood',
 'numbad',
 'ndf',
 'total',
 'numpix',
 'skewness',
 'order',
 'mincoord',
 'maxwcs',
 'maxpos')

## Getting help

If we can't remember which Starlink we are running, we can check by looking at:

In [8]:
kappa.wrapper.starpath

'/star'

If you need to look at the call signature and parameter info of a command, you can use the standard help() function.

In [9]:
help(kappa.stats)

Help on function stats in module starlink.kappa:

stats(ndf, **kwargs)
    Computes simple statistics for an NDF's pixels.
    
    Runs the command: $KAPPA_DIR/stats .
    
    Arguments
    ---------
    ndf : str,filename
        Data structure to analyse
    
    
    Keyword Arguments
    -----------------
    comp : str
        NDF array component to analyse ["Data"]
    
    clip : List[float]
        Clipping levels [!]
    
    logfile : str,filename
        File for logging results [!]
    
    order : bool
        Report order statistics? [FALSE]
    
    percentiles : List[float]
        List of percentiles [!]
    
    
    Returns
    -------
    kurtosis : float
    
    maxcoord : List[float]
    
    maximum : float
    
    maxpos : List[int]
    
    maxwcs : str
    
    mean : float
    
    median : float
    
    mincoord : List[float]
    
    minimum : float
    
    minpos : List[int]
    
    minwcs : str
    
    numbad : int
    
    numgood : int
    
    

Note that at the bottom it provides a link to the web documentation for that command. Alternatively, you can look at the SUNs and commandline help that come with your Starlink installation.

There's also a special utilities module which contains a ```starhelp``` command which for modules will show you the listing of commands and their purpose for that module, or for commands will show you the full (often extremely long) RST formatted version of the original Starlink help for that command. This is shown below for the CUPID package (which doesn't have very many commands.)

In [9]:
from starlink import utilities, cupid

In [10]:
utilities.starhelp(cupid)

clumpinfo     :     Obtain information about one or more previously identified clumps.
cupidhelp     :     Display information about CUPID.
extractclumps :     Extract previously identified clumps of emission from an NDF.
findback      :     Estimate the background in an NDF by removing small scale structure.
findclumps    :     Identify clumps of emission within a 1, 2 or 3 dimensional NDF.
makeclumps    :     Create simulated data containing clumps and noise.
outlineclump  :     Draw an outline around a 2-dimensional clump identified by CUPID.

In [12]:
utilities.starhelp(cupid.clumpinfo)



CLUMPINFO


Purpose
~~~~~~~
Obtain information about one or more previously identified clumps


Description
~~~~~~~~~~~
This application returns various items of information about a single
clump, or a collection of clumps, previously identified using
FINDCLUMPS or EXTRACTCLUMPS.


Usage
~~~~~


::

    
       clumpinfo ndf clumps quiet
       



ADAM parameters
~~~~~~~~~~~~~~~



CLUMPS = LITERAL (Read)
```````````````````````
Specifies the indices of the clumps to be included in the returned
information. It can take any of the following values:


+ "ALL" or "*" -- All clumps.
+ "xx,yy,zz" -- A list of clump indices.
+ "xx:yy" -- Clump indices between xx and yy inclusively. When xx is
omitted the range begins from one; when yy is omitted the range ends
with the final clump index.
+ Any reasonable combination of above values separated by commas.





FLBND( ) = _DOUBLE (Write)
``````````````````````````
The lower bounds of the bounding box enclosing the selected clumps in
the curren

## Running more complex commands

You can also run more complex commands, such as those that produce new output files, such as CUPID's ```findclumps```. For example:

In [13]:
from starlink import cupid

In [16]:
g34map = '/export/data/sgraves/M16AD003/g34-3-tangent_30318.sdf'

Because Starlink has a very complex method for automatically determining defaults which doesn't map very easily to pythonic required argument and optional keyword commands, you'll probably find that when you first run a command, you get a lot of errors about missing parameters, like this:

In [17]:
cupid.findclumps(g34map, 'g34-clumpmask', outcat='g34clumpcat', method='fellwalker', backoff=False)

Exception: Starlink error occured during command:
/star/bin/cupid/findclumps ('/export/data/sgraves/M16AD003/g34-3-tangent_30318.sdf', 'g34-clumpmask')
 stdout and stderr are appended below.

!! SUBPAR: Parameter RMS has no value - prompting disallowed.
!  FINDCLUMPS: Failed to identify clumps of emission within a 1, 2 or 3-D
!     NDF.
!  Application exit status PAR__NOUSR, No user available for input
!  /export/data/sgraves/M16AD003/g34-3-tangent_30318.sdf g34-clumpmask
!     method=fellwalker outcat=g34clumpcat backoff=False



In this case, it can be fixed by supplying a value for the RMS. (If you've used CUPID on the command line, you'll know that while it can guess the appropriate level for the RMS by looking at the noise map of the input NDF, it only does this via interactive prompting. As its not possible to prompt the user within a script, you'll instead have to supply this value yourself.)

In [20]:
clumpresults = cupid.findclumps(g34map, 'g34-clumpmask', outcat='g34clumpcat.FIT', method='fellwalker',
                                backoff=False, RMS=0.2)

In [21]:
clumpresults

    in_    b'/export/data/sgraves/M16AD003/g34-3-tangent_30318.sdf'
    rms    0.2
    out    b'g34-clumpmask'
nclumps    63
 outcat    b'g34clumpcat.FIT'
repconf    1
backoff    0
 deconv    1
 wcspar    1
 method    b'fellwalker'
  shape    b'NONE'

This tells you that you find 63 clumps, they are stored in a ```g34-clumpmask.sdf``` file, you used the fellwalker method, and you made an output catalogue name ```g34clumpcat.FIT```.

# Problems and Issues

## Known issues.
Currently, the SMURF scripts which are really python scripts maskerading as Starlink commands (pol2scan, pol2map, most pol2... commands, jsaplit and a few others) won't raise an exit error if they encounter a problem. As a result, if you 


## Accessing the stdout.
If you set the logging level to 'DEBUG' (see above), you'll see the standard output from every command reported on the screen. If you also need access to the standard output as a string, pass the keyword argument ```returnStdOut=True``` to any of the commands. This will make the command return two values: the normal namedtuple object with the output parameters, and the standard output as a string.

Unfortunately the std output won't appear until the command finishes -- this will be primarily an issue for the long commands, mainly makemap, makecube, pol2scan and pol2map, and the oracdr recipes.

## Issues with starlink-pyndf

starlink-pyndf is used to access the output values from Starlink commands, and it usually requires the LD_LIBRARY_PATH (Linux) or ```$DYLD_LIBRARY_PATH``` environemntal variable to include the location of the various hds/ndf Starlink libraries. Normally this can be done by setting DYLD_LIBRARY_PATH (or LD_LIBRARY_PATH) to point to the ```$STARLINK_DIR/lib``` location before you start up python.

If you haven't done that, you'll probably see an error like:

In [1]:
from starlink import wrapper

ImportError: dlopen(/usr/local/lib/python3.5/site-packages/starlink/hds.cpython-35m-darwin.so, 2): Library not loaded: @rpath/lib/libndf.0.dylib
  Referenced from: /usr/local/lib/python3.5/site-packages/starlink/hds.cpython-35m-darwin.so
  Reason: image not found

## What if I can't run the command because its asking for the wrong parameters?

If the autogeneration goes wrong, it could make it impossible to run one of the commands using the provided function python function. Please let me know if that happens, but if you need to work around it you can use the ```starlink.wrapper.starcomm``` command to run the command instead. This will let you run the command by specifying the command's path (which can include the standard Starlink environmental variables like $KAPPA_DIR), its name and its arguments as string.

In [1]:
from starlink import wrapper

In [2]:
help(wrapper.starcomm)

Help on function starcomm in module starlink.wrapper:

starcomm(command, commandname, *args, **kwargs)
    Execute a Starlink application
    
    Carries out the starlink command, and returns a namedtuple of the
    starlink parameter values (taken from $ADAM_DIR/<com>.sdf
    
    Arguments
    ---------
    command: str
       The path of a command to run, e.g. '$SMURFDIR/makecube'
    
    commandname: str
       The name of command (used for getting output values).
    
    Keyword arguments
    -----------------
    returnstdout: bool
    
        Have the string that would have been written to stdout in a
        normal starlink session be returned from this function as a
        string.
    
    
    Other arguments and keyword arguments are evaluated by the command
    being called. Please see the Starlink documentation for the command.
    The standard Starlink package environmental variables (e.g. KAPPA_DIR,
    SMURF_DIR etc.) can be used inside the command name.
    
    R